<span style="font-size: 38px; color: Green; font-weight: bold;">"Where Should I Live"</span><br>
<span style="font-size: 28px; color: Orange; font-weight: bold;">Group Project 
Programming for Data Science 2025/26
</span>


<span style="font-size: 28px; color: Red; font-weight: bold;">LCD_PDS_2526_Group22</span>

<span style="font-size: 18px; color: Purple; font-weight: bold;">Student Id:20251000 Student Name: Atila Soleimanian </span><br>
<span style="font-size: 18px; color: Purple; font-weight: bold;">Student Id:20241027 Student Name: Ali Fardin Gulistani</span><br>
<span style="font-size: 18px; color: Purple; font-weight: bold;">Student Id:20241831 Student Name: Daniyal Ahmad </span><br>
<span style="font-size: 18px; color: Purple; font-weight: bold;">Student Id:20241418  Student Name: A K M Saif Hoque</span>

# STEP 1: IMPORTING DATASET & LIBRARIES 

In [408]:
#!pip install pandas (In case if the pandas show eror uncomment by deleting hashtag and also deleting the texts in bracket and install pandas)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [409]:
df=pd.read_csv('city_data.csv',sep="|") #provided file uses pipe '|' as delimiter, so the dataset is separated from the delimiter '|'.

In [410]:
df.head(2)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
1,"Vienna, Austria",310,2983513,2018818,20.1,10.2,55770,3,"German, English, Turkish, Serbian",2500,1050,2061,340,2024-06-15 00:00:00


### As we can see, the first row of the dataset doesn't apply any importance as the names of the features are described in the 2nd row. So we will remove the first row using ***header***

In [411]:
df=pd.read_csv('city_data.csv',sep="|",header=1)

In [412]:
df.head(2)

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Avgerage Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
0,"Vienna, Austria",310.0,2983513,2018818.0,20.1,10.2,55770.0,3,"German, English, Turkish, Serbian",2500,1050,2061,340.0,2024-06-15 00:00:00
1,"Salzburg, Austria",243.0,375489,250472.0,20.3,3.0,66689.0,0,German,3200,1100,2186,NaN,2023-11-03 00:00:00


# STEP 2: STATISTICS OF THE DATASET

In [413]:
print(f'The shape of the dataset is {df.shape}')     

The shape of the dataset is (86, 14)


In [414]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   City                             86 non-null     object 
 1   Population Density               85 non-null     float64
 2   Population                       86 non-null     int64  
 3   Working Age Population           85 non-null     float64
 4   Youth Dependency Ratio           86 non-null     float64
 5   Unemployment Rate                85 non-null     float64
 6   GDP per Capita                   85 non-null     float64
 7   Days of very strong heat stress  86 non-null     int64  
 8   Main Spoken Languages            85 non-null     object 
 9   Average Monthly Salary           86 non-null     int64  
 10  Avgerage Rent Price              86 non-null     int64  
 11  Average Cost of Living           86 non-null     int64  
 12  Average Price Groceries 

In [415]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Population Density,85.0,1.115141e+03,2.542779e+03,65.00,304.000,548.00,1020.000,20983.0
Population,86.0,1.754370e+06,1.843133e+06,73895.00,528118.500,1178397.00,2315896.000,12396541.0
Working Age Population,85.0,1.158294e+06,1.232534e+06,46359.00,358020.000,733794.00,1534225.000,8274794.0
Youth Dependency Ratio,86.0,2.409686e+01,3.266421e+00,19.30,21.425,23.15,25.925,37.8
Unemployment Rate,85.0,6.667059e+00,4.553632e+00,1.40,3.700,5.30,8.100,23.0
GDP per Capita,85.0,5.175027e+04,2.149562e+04,10089.92,38664.000,49329.00,63083.000,146094.0
Days of very strong heat stress,86.0,4.348837e+00,1.003848e+01,0.00,0.000,1.00,3.000,59.0
Average Monthly Salary,86.0,2.434488e+03,1.188603e+03,750.00,1412.500,2400.00,3200.000,6636.0
Avgerage Rent Price,86.0,1.070105e+03,4.989646e+02,100.00,800.000,1050.00,1250.000,3190.0
Average Cost of Living,86.0,1.808384e+03,7.279997e+02,800.00,1300.000,1665.00,2100.000,4862.0


# STEP 3: DATA TYPES & CONVERSION 

 ### From the description of the dataset, it's clearly visible that most of the features don't need **float64** as a datatype. So the datatypes are converted below as their need

In [416]:
df = df.astype({'Population Density' : 'float32', 
                'Population': 'float32',
                'Youth Dependency Ratio': 'float32',
                'Unemployment Rate': 'float32',
                'GDP per Capita': 'float32',
                'Days of very strong heat stress':'float32',
                'Average Monthly Salary':'float32',
                'Avgerage Rent Price':'float32',
                'Average Price Groceries':'float32',
                'Average Cost of Living':'float32'
                })

 CHANGING <span style="font-size: 23px; color: Green;font-weight: bold ;">"Last Date Update "</span> column datatype ***object*** into ***datetime*** datatype.

In [417]:
df['Last Data Update'] = pd.to_datetime(df['Last Data Update'])
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   City                             86 non-null     object        
 1   Population Density               85 non-null     float32       
 2   Population                       86 non-null     float32       
 3   Working Age Population           85 non-null     float64       
 4   Youth Dependency Ratio           86 non-null     float32       
 5   Unemployment Rate                85 non-null     float32       
 6   GDP per Capita                   85 non-null     float32       
 7   Days of very strong heat stress  86 non-null     float32       
 8   Main Spoken Languages            85 non-null     object        
 9   Average Monthly Salary           86 non-null     float32       
 10  Avgerage Rent Price              86 non-null     float32       


### As we can see, nearly 10 KB has been less in memory usage for data conversion.

# STEP 4 DATA QUALITY CHECK 
## ***4.1>MISSING VALUE*** 
## ***4.2>DUPLICATE VALUE***
## ***4.3>VALUE RANGE CHECK***

## 4.1> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">MISSING VALUE</span>

In [418]:
missing_data = df.isnull().sum().reset_index()
missing_data.columns = ['Column', 'MissingCount']
missing_data['MissingPercent'] = (missing_data['MissingCount'] / len(df)) * 100
missing_data.sort_values(by='MissingPercent',ascending=False)

,Column,MissingCount,MissingPercent
12,Average Price Groceries,80,93.023256
1,Population Density,1,1.162791
3,Working Age Population,1,1.162791
5,Unemployment Rate,1,1.162791
6,GDP per Capita,1,1.162791
8,Main Spoken Languages,1,1.162791
0,City,0,0.000000
2,Population,0,0.000000
4,Youth Dependency Ratio,0,0.000000
7,Days of very strong heat stress,0,0.000000


## Missing Value analysis 
- As we can see, out of 86 entries in the dataset, there are 80 missing values in the **Average Price Groceries** column, which has more than 93% missing values in the dataset.


## 4.2> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">DUPLICATED VALUE</span>

In [419]:
duplicate_rows = df[df.duplicated(keep=False)]
print(f'Total number of duplicate rows: {len(duplicate_rows)}')

Total number of duplicate rows: 4


In [420]:
print(" Duplicate Rows  Are:")
print(duplicate_rows[['City', 'Population', 'Average Monthly Salary', 
                          'Last Data Update']])

 Duplicate Rows  Are:
               City  Population  Average Monthly Salary Last Data Update
15  Berlin. Germany   5303922.0                  3200.0       2023-06-29
16  Berlin. Germany   5303922.0                  3200.0       2023-06-29
32    Malaga, Spain    869096.0                  2200.0       2023-11-27
33    Malaga, Spain    869096.0                  2200.0       2023-11-27


- As we can see, there are 4 rows with 2 identical values. 

## 4.3> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">VALUE RANGE DETECTION</span>

In [421]:
print("### VALUE RANGE VALIDATION ###\n")

if df['Population'].min() < 0:
    print("Population has negative values")
else:
    print("Population Ok (all positive)")

un_min = df['Unemployment Rate'].min()
un_max = df['Unemployment Rate'].max()
if un_min < 0 or un_max > 100:
    print(f' Unemployment Rate Outside 0–100: {un_min}% to {un_max}%')
else:
    print(f'Unemployment Rate Ok: {un_min}% to {un_max}% . So,Unemployment rate within valid range')


yd_min = df['Youth Dependency Ratio'].min()
yd_max = df['Youth Dependency Ratio'].max()
if yd_min < 0 or yd_max > 100:
    print(f' Youth Dependency Ratio Outside 0–100: {yd_min}% to {yd_max}%')
else:
    print(f' Youth Dependency Ratio Ok: {yd_min}% to {yd_max}%')


sal_min = df['Average Monthly Salary'].min()
if sal_min < 0:
    print(' Salary has negative values')
else:
    print('Salary Ok (all positive)')


### VALUE RANGE VALIDATION ###

Population Ok (all positive)
Unemployment Rate Ok: 1.399999976158142% to 23.0% . So,Unemployment rate within valid range
 Youth Dependency Ratio Ok: 19.299999237060547% to 37.79999923706055%
Salary Ok (all positive)


# STEP 5 DATA CLEANING & ANSWER TO Q1 
## ***5.1>FIXING TYPO*** 
## ***5.2>FIXING CITY FEATURE***
## ***5.3> HANDLING DUPLICATE VALUE***(Q1)
## ***5.4>HANDLING MISSING VALUE***(Q1)

## 5.1> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">FIXING TYPO</span>

In [422]:
df.columns.tolist()

['City',
 'Population Density',
 'Population',
 'Working Age Population ',
 'Youth Dependency Ratio',
 'Unemployment Rate',
 'GDP per Capita',
 'Days of very strong heat stress',
 'Main Spoken Languages',
 'Average Monthly Salary',
 'Avgerage Rent Price',
 'Average Cost of Living',
 'Average Price Groceries',
 'Last Data Update']

-As we can see in the <span style="font-size: 15px; color:RED;font-weight: bold ;">'Avgerage Rent Price'</span> there is a typing mistake which is ***avgerage*** and we need to change it into ***Average***

In [423]:
df.rename(columns={'Avgerage Rent Price': 'Average Rent Price'}, inplace=True)
df.head(1)

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update
0,"Vienna, Austria",310.0,2983513.0,2018818.0,20.1,10.2,55770.0,3.0,"German, English, Turkish, Serbian",2500.0,1050.0,2061.0,340.0,2024-06-15


In [424]:
print('___________Before_____________')
df.iloc[45]['City']

___________Before_____________


'Greece, Athens'

- As we can see in the <span style="font-size: 15px; color:RED;font-weight: bold ;">***City***</span> feature in the 45th row there is a mistake that the name of the country is written first and name of the city after comma which is unsual then other examples in the ***City*** feature.
- We will fix this by replacing <span style="font-size: 15px; color:RED;font-weight: bold ;">'Greece, Athens'</span> with <span style="font-size: 15px; color:green;font-weight: bold ;">'Athens, Greece'</span>.

In [425]:
df.iloc[45,0]='Athens, Greece'

In [426]:
print('___________After_____________')
df.iloc[45]['City']

___________After_____________


'Athens, Greece'

## 5.1> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">FIXING CITY FEATURE</span>

- In the ***City*** feature, there is a city and a country described in the same feature by separating them with (, . ;). So we will create 2 new columns named ***City_name*** and ***Country*** by splitting them from the City feature for better exploration.

In [427]:
df[['City_Name', 'Country']] = (
    df['City']
    .str.split(r'[,;.]\s*',expand=True)
)

In [428]:
df.head(5)

,City,Population Density,Population,Working Age Population,Youth Dependency Ratio,Unemployment Rate,GDP per Capita,Days of very strong heat stress,Main Spoken Languages,Average Monthly Salary,Average Rent Price,Average Cost of Living,Average Price Groceries,Last Data Update,City_Name,Country
0,"Vienna, Austria",310.0,2983513.0,2018818.0,20.100000,10.2,55770.0,3.0,"German, English, Turkish, Serbian",2500.0,1050.0,2061.0,340.0,2024-06-15,Vienna,Austria
1,"Salzburg, Austria",243.0,375489.0,250472.0,20.299999,3.0,66689.0,0.0,German,3200.0,1100.0,2186.0,NaN,2023-11-03,Salzburg,Austria
2,"Brussels, Belgium",681.0,3284548.0,2137425.0,27.500000,10.7,62500.0,3.0,"French, Dutch, Arabic, English",3350.0,1200.0,1900.0,NaN,2023-04-22,Brussels,Belgium
3,"Antwerp, Belgium",928.0,1139663.0,723396.0,27.700001,6.2,57595.0,3.0,"Dutch, French, Arabic",2609.0,900.0,1953.0,NaN,2024-08-09,Antwerp,Belgium
4,"Gent, Belgium",552.0,645813.0,417832.0,24.799999,NaN,53311.0,2.0,"Dutch, French",2400.0,827.0,1200.0,120.0,2023-07-17,Gent,Belgium


- So, we don't need this city feature for now as we have ***City_Name*** and ***Country*** feature.

In [429]:
df= df.drop(columns=['City'])

## 5.3> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">HANDLING DUPLICATE VALUE</span><span style="font-size: 23px; color:GREEN;font-weight: bold ;"> (Q1)</span>

In [430]:
df_clean = df.copy()

In [431]:
rows_before = len(df_clean)
df_clean = df_clean.drop_duplicates()

rows_after = len(df_clean)
duplicates_removed = rows_before - rows_after
print(f"Duplicates removed: {duplicates_removed}")

Duplicates removed: 2


In [432]:
df_clean = df_clean.reset_index(drop=True)

- As we removed 2 of the duplicated rows from the dataset, their index were also removed from the dataset. So we need to reset the index .

## 5.4> <span style="font-size: 23px; color:ORANGE;font-weight: bold ;">HANDLING MISSING VALUE</span><span style="font-size: 23px; color:GREEN;font-weight: bold ;">(Q1)</span>

In [433]:
df_clean.isna().sum()

Population Density                  1
Population                          0
Working Age Population              1
Youth Dependency Ratio              0
Unemployment Rate                   1
GDP per Capita                      1
Days of very strong heat stress     0
Main Spoken Languages               1
Average Monthly Salary              0
Average Rent Price                  0
Average Cost of Living              0
Average Price Groceries            78
Last Data Update                    0
City_Name                           0
Country                             0
dtype: int64

## As we can see, there are still 78 missing values in ***Average Price Groceries*** Column. So, we will drop this feature 

In [434]:
df_clean = df_clean.drop(columns=['Average Price Groceries'])
df_clean.isna().sum()

Population Density                 1
Population                         0
Working Age Population             1
Youth Dependency Ratio             0
Unemployment Rate                  1
GDP per Capita                     1
Days of very strong heat stress    0
Main Spoken Languages              1
Average Monthly Salary             0
Average Rent Price                 0
Average Cost of Living             0
Last Data Update                   0
City_Name                          0
Country                            0
dtype: int64

## Now, we have 5 features with missing values 
- Population Density
- Working Age Population
- Unemployment Rate
- GDP per Capita
- Main Spoken Languages
### We will fill in the missing values using various methods.

# FILLING <span style="font-size: 23px; color:YELLOW;font-weight: bold ;">***WORKING AGE POPULATION*** </span>FEATURE missing value

In [435]:
missing_working_age = df_clean['Working Age Population '].isna()
missing_working_age.sum()

np.int64(1)

In [436]:
print('..................BEFORE..............')


print(df_clean[missing_working_age][['City_Name','Country', 'Population', 'Working Age Population ']])

..................BEFORE..............
     City_Name  Country  Population  Working Age Population 
23  Dusseldorf  Germany   1554077.0                      NaN


- As we can see, there is 1 missing value in this feature, and we are going to fill this by taking the median ratio between the feature and ***population*** 

In [437]:
print('..................AFTER..............')


working_age_ratio = (df_clean['Working Age Population '] / df_clean['Population']).median()
print(f"Median ratio of Working Age to Total Population: {working_age_ratio}%")
df_clean.loc[missing_working_age, 'Working Age Population '] = (df_clean.loc[missing_working_age, 'Population'] * working_age_ratio).round()
print(df_clean[missing_working_age][['City_Name', 'Country','Population', 'Working Age Population ']])

..................AFTER..............
Median ratio of Working Age to Total Population: 0.6589942050401019%
     City_Name  Country  Population  Working Age Population 
23  Dusseldorf  Germany   1554077.0                1024128.0


In [438]:
df_clean.loc[23, 'Working Age Population ']

np.float64(1024128.0)

## In this case, we are taking the median of the ratio between ***Working Age Population*** and ***Population***. 
## At first, look at rows where the Working Age Population is missing
## Then take the population of that row
## Then Multiply population × ratio
## At last, we fill the missing values 

# FILLING <span style="font-size: 23px; color:YELLOW;font-weight: bold ;">***Unemployment Rate*** </span>FEATURE missing value

In [439]:
missing_unemployment_rate=df_clean['Unemployment Rate'].isna()

In [440]:
print(df_clean[missing_unemployment_rate][['City_Name','Country', 'Population', 'Unemployment Rate' ]])

  City_Name  Country  Population  Unemployment Rate
4      Gent  Belgium    645813.0                NaN


In [441]:
df_clean['Unemployment Rate'] = df_clean['Unemployment Rate'].interpolate(method='linear')
print(df_clean[missing_unemployment_rate][['City_Name', 'Country','Population', 'Unemployment Rate' ]])

  City_Name  Country  Population  Unemployment Rate
4      Gent  Belgium    645813.0                5.9


In [442]:
df_clean.loc[ 4,'Unemployment Rate']

np.float32(5.8999996)

## In this case, we are using the linear interpolation method, which fills missing values by creating a value between the numbers above and below the missing point.

# FILLING <span style="font-size: 23px; color:YELLOW;font-weight: bold ;">***GDP per Capita*** </span>FEATURE missing value

In [443]:
missing_gdp = df_clean['GDP per Capita'].isna()
missing_count = missing_gdp.sum()

for idx in df_clean[missing_gdp].index:
    country = df_clean.loc[idx, 'Country']
    print('__________Before__________')
    print(df_clean.loc[idx, ['City_Name', 'Country', 'GDP per Capita']])  
        
    country_gdp_mean = df_clean[df_clean['Country'] == country]['GDP per Capita'].mean()
    df_clean.loc[idx, 'GDP per Capita'] = country_gdp_mean
    print('__________After__________')  
    print(df_clean.loc[idx, ['City_Name', 'Country', 'GDP per Capita']])

__________Before__________
City_Name         Lemesos
Country            Cyprus
GDP per Capita        NaN
Name: 12, dtype: object
__________After__________
City_Name         Lemesos
Country            Cyprus
GDP per Capita    34907.0
Name: 12, dtype: object


- For filling the missing value, here we go through each row to check where the missing value is and create a true/false table by ***df_clean['GDP per Capita'].isna()*** and store it into ***missing_gdp***.
- ***missing_count = missing_gdp.sum()*** counts how many values are true, which means how many missing values there are.
- Then we use a for loop to select all rows where ***GDP per Capita*** is missing and loop through their indexes.
- ***country = df_clean.loc[idx, 'Country']*** by this code, we go to the country where there is a missing value, and here we found that there is a missing value in the country **Cyprus**.
- ***country_gdp_mean = df_clean[df_clean['Country'] == country]['GDP per Capita'].mean()*** This code filters all rows from **Cyprus** country and takes the mean ***GDP per Capita*** of  **Cyprus**.
- ***df_clean.loc[idx, 'GDP per Capita'] = country_gdp_mean*** This code fills the missing value with the mean of **Cyprus**.

# FILLING <span style="font-size: 23px; color:YELLOW;font-weight: bold ;">***Population Density*** </span>FEATURE missing value

In [444]:
missing_density = df_clean['Population Density'].isna()
missing_count = missing_density.sum()
for idx in df_clean[missing_density].index:
    country = df_clean.loc[idx, 'Country']
    print('__________Before__________')
    print(df_clean.loc[idx, ['City_Name', 'Country', 'Population Density']])
        
    country_density_median = df_clean[df_clean['Country'] == country ]['Population Density'].median()
    df_clean.loc[idx, 'Population Density'] = country_density_median
    print('__________After__________')   
    print(df_clean.loc[idx, ['City_Name', 'Country', 'Population Density']])

__________Before__________
City_Name             Valencia
Country                  Spain
Population Density         NaN
Name: 29, dtype: object
__________After__________
City_Name             Valencia
Country                  Spain
Population Density       722.0
Name: 29, dtype: object


- For filling the missing value, here we go through each row to check where the missing value is and create a true/false table by ***df_clean['Population Density'].isna()*** and store it into ***missing_density***.
- ***missing_count = missing_density.sum()*** counts how many values are true, which means how many missing values there are.
- Then we use a for loop to select all rows where ***'Population Density'*** is missing and loop through their indexes.
- ***country = df_clean.loc[idx, 'Country']*** by this code, we go to the country where there is a missing value, and here we found that there is a missing value in the country **Spain**.
- ***country_density_median = df_clean[df_clean['Country'] == country ]['Population Density'].median()*** This code filters all rows from **Spain** country and takes the median ***'Population Density'*** of  **Spain**.
- ***df_clean.loc[idx, 'Population Density'] = country_density_median*** This code fills the missing value with the median of **Spain**.

# FILLING <span style="font-size: 23px; color:YELLOW;font-weight: bold ;">***Main Spoken Languages*** </span>FEATURE missing value.

In [445]:
missing_languages = df_clean['Main Spoken Languages'].isna()
missing_count = missing_languages.sum()

for idx in df_clean[missing_languages].index:
    country = df_clean.loc[idx, 'Country']
        
    print('__________Before__________')
    print(df_clean.loc[idx, ['City_Name', 'Country', 'Main Spoken Languages']])
        
    country_language_mode = df_clean[df_clean['Country'] == country]['Main Spoken Languages'].mode()
    df_clean.loc[idx, 'Main Spoken Languages'] = country_language_mode[0]
    print('__________After__________')
    print(df_clean.loc[idx, ['City_Name', 'Country', 'Main Spoken Languages']])

__________Before__________
City_Name                Dusseldorf
Country                     Germany
Main Spoken Languages           NaN
Name: 23, dtype: object
__________After__________
City_Name                Dusseldorf
Country                     Germany
Main Spoken Languages        German
Name: 23, dtype: object


- For filling the missing value, here we go through each row to check where the missing value is and create a true/false table by *** ***df_clean['Main Spoken Languages'].isna()*** and store it into ***missing_languages***.
- ***missing_count = missing_languages.sum()*** counts how many values are true, which means how many missing values there are.
- Then we use a for loop to select all rows where ***' Main Spoken Languages'*** is missing and loop through their indexes.
- ***country = df_clean.loc[idx, 'Country']*** by this code, we go to the country where there is a missing value, and here we found that there is a missing value in the country **Germany**.
- ***country_language_mode = df_clean[df_clean['Country'] == country]['Main Spoken Languages'].mode()*** This code filters all rows from **Germany** country and look at their languages and takes the mode ***''Main Spoken Languages'*** of  **Germany** / find the most frequent language of ***Germany***.
- ***df_clean.loc[idx, 'Main Spoken Languages'] = country_language_mode[0]*** This code fills the missing value with the median of **Germany**.

## <h1 style="text-align: center;">Summary for Q1</h1>
- For handling ***Duplicated Values***, we removed both the duplicated values .
- For handling ***Missing Values***, we removed **'Average Price Groceries'** feature as it had 93% missing values.
- For the other 5 features with missing values, we used different methods.
| Feature    | Used Method   | Mehod used in      |
|:---------|:---------:|--------------:|
|  Main Spoken Languages | Mode  | used only in the country where there was a missing value    |
| Population Density   | Median |    used only in the country where there was a missing value  |
| PGDP per Capita  | Mean |    used only in the country where there was a missing value  |
| Unemployment Rate  | Linear |    used linear interpolation in the next and previous values of the missing value  |
| Working Age Population  | Median |  used the median of the ratio between Working Age Population and Population, and then multiplied it by the population of the row where there was a missing value and filled it    |
